# Simple Naive RAG within BigQuery

## Configurations

In [ ]:
project_id = "<YOUR PROJECT ID here>"

In [ ]:
project_number = !(gcloud projects describe $project_id --format="value(projectNumber)")
project_number=project_number[0]

In [ ]:
project_number

In [ ]:
!pip install -q datasets

In [ ]:
# @title Linking BigQuery to Pandas with _BigFrames_

import bigframes.pandas as bf

bf.options.bigquery.location = "us" #this variable is set based on the dataset you chose to query
bf.options.bigquery.project = project_id

In [ ]:
from datasets import load_dataset

## Creating a corpus

In [ ]:
rag_corpus_sample = load_dataset("rag-datasets/mini_wikipedia", "text-corpus", split = "passages[:10%]")

In [ ]:
corpus_df = rag_corpus_sample.to_pandas()

In [ ]:

corpus_df.head()

In [ ]:

from bigframes.ml.llm import PaLM2TextEmbeddingGenerator
model = PaLM2TextEmbeddingGenerator()


def embed_column_from_df(df, column):
    df1 = df.rename(columns = {column:'content'})[['content']]
    bf1 = bf.DataFrame(df1.to_dict(orient='records'))
    return model.predict(bf1)

In [ ]:
corpus_embeddings = embed_column_from_df(corpus_df, "passage")


In [ ]:
corpus_embeddings[["text_embedding", "content"]]

In [ ]:
corpus_embeddings.to_gbq(destination_table="cloud_embeddings.corpus", if_exists="replace")

### Create a remote connection

In [ ]:
connection_id = "demo_rag"
full_connection_id = f"{project_id}.us.{connection_id}"

full_connection_id

In [ ]:
!bq mk --connection --location=us --project_id=$project_id --connection_type=CLOUD_RESOURCE $connection_id

In [ ]:
!bq ls --connection --project_id=$project_id --location=us

In [ ]:
!bq --format=json --location us show --connection $connection_id

In [ ]:
connectionConfig[0]


In [ ]:
import json
connectionConfig=!bq --format=json --location us show --connection $connection_id
print(connectionConfig)
service_account_id = json.loads(connectionConfig[0])['cloudResource']['serviceAccountId']
service_account_id

In [ ]:
member=f'serviceAccount:{service_account_id}'
member

### Authorize remote connection to use models



In [ ]:
!gcloud projects add-iam-policy-binding $project_number --member=$member --role='roles/aiplatform.user' --condition=None

In [ ]:
embedding_name = "mygecko"
dataset = "cloud_embeddings"
full_embedding_name = f"{project_id}.{dataset}.{embedding_name}"
full_embedding_name

In [ ]:
full_connection_id

In [ ]:
%%bigquery --params $params

CREATE MODEL `EMBEDDING_FULL_NAME`
REMOTE WITH CONNECTION `YOUR_REMOTE_CONNECTION`
OPTIONS(ENDPOINT = "textembedding-gecko@003");

In [ ]:
### Retriever

In [ ]:

%%bigquery results

WITH query_embedding AS (
  SELECT
    *
  FROM
    ML.GENERATE_TEXT_EMBEDDING(MODEL cloud_embeddings.mygecko,
    (
    SELECT
      "What is the capital of Uruguay?" AS content))
)
SELECT
  c.content AS candidate_text,
FROM
  query_embedding AS q,
  cloud_embeddings.corpus AS c
ORDER BY
    ML.DISTANCE(q.text_embedding, c.text_embedding, 'COSINE')
 ASC
LIMIT 20;

In [ ]:
results

### Retriever part

In [ ]:
%%bigquery results

WITH query_embedding AS (
  SELECT
    *
  FROM
    ML.GENERATE_TEXT_EMBEDDING(MODEL cloud_embeddings.mygecko,
    (
    SELECT
      "What is the capital of Uruguay?" AS content))
)
SELECT
  q.content AS query_text,
  c.content AS candidate_text,
  ML.DISTANCE(q.text_embedding, c.text_embedding, 'COSINE') AS distance
FROM
  query_embedding AS q,
  cloud_embeddings.corpus AS c
ORDER BY
  distance ASC
LIMIT 20;

In [ ]:
results

### Generator part

In [ ]:
%%bigquery


CREATE MODEL `YOUR_GENERATOR_NALME`
REMOTE WITH CONNECTION `CONNECTION_ID`
OPTIONS(ENDPOINT="gemini-pro");

In [ ]:
%%bigquery

WITH initial_query as

(SELECT "What is the capital of Tunisia?" as content)


, query_embedding AS (
  SELECT
    *
  FROM
    ML.GENERATE_TEXT_EMBEDDING(MODEL cloud_embeddings.mygecko,(SELECT content from initial_query))),

ranked_docs as
(SELECT
  q.content AS query_text,
  c.content AS candidate_text,
  ML.DISTANCE(q.text_embedding, c.text_embedding, 'COSINE') AS distance
FROM
  query_embedding AS q,
  cloud_embeddings.corpus AS c
ORDER BY
  distance ASC
LIMIT 10),

curated_docs as
(select string_agg(candidate_text) as documents from ranked_docs),

final_prompt as

(select concat("Answer concisely to the following question <question>:\n",
        content, "</question>",
        "\n only and only with the help of the following documents: <documents>", curated_docs.documents, "<documents>",
        "Don't try to make up an answer if it is not shown in the provided documents") as prompt from curated_docs, initial_query)


SELECT
  ml_generate_text_result['predictions'][0]['content'] AS generated_text
  --,ml_generate_text_result['predictions'][0]['safetyAttributes']
 --   AS safety_attributes,
  --* EXCEPT (ml_generate_text_result)

 from ML.GENERATE_TEXT(MODEL `cloud_embeddings.mygemini`, (select prompt from final_prompt), STRUCT( 0.1 as temperature, 500 as max_output_tokens) )
